Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [311]:
from random import choices, choice, randint, random
from dataclasses import dataclass
from copy import copy, deepcopy

import lab9_lib

In [63]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11111000010000001011100100110000000101010011101000: 17.56%
10111000101010101000010011010110011100110001000110: 7.33%
00001011001001010001011011110001101101101100000001: 23.56%
11110101100000110110000111101011010000110101110100: 23.33%
01010111000111100111111011010010100111010011000000: 15.33%
10011011000001010001111000000110101101101000110011: 7.33%
01100011100001111010100110101101110110001110111010: 9.13%
01100010100011101101001001001010000111110011100111: 15.33%
00101000010110000011111011101000011100010110000000: 7.33%
01101001010010101011111110111110110111011010011101: 19.11%
10


In [314]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 50
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = 0.15

NUM_SETS = 1000

fitness = lab9_lib.make_problem(10)

@dataclass
class Individual:
    fitness: float
    genotype: list[int]

def populate():
    population = [
        Individual(
            genotype=[choice((0,1)) for _ in range(NUM_SETS)],
            fitness=None,
        )
        for _ in range(POPULATION_SIZE)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)

    return population

def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual, population) -> Individual:
    offspring = copy(ind)

    best = NUM_SETS
    index = -1
    for i in range(NUM_SETS):
        counter = 0
        for j in range(len(population)):
            if population[j].genotype[i] == 1:
                counter += 1

        if counter < best:
            best = counter
            index = i

    if index != -1:
        pos = index
    else:
        pos = randint(0, NUM_SETS-1)

    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutate2(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def my_xover(ind1: Individual, ind2: Individual) -> Individual:
    list = [0 for _ in range(NUM_SETS)]

    for i in range(NUM_SETS):
        list[i] = ind1.genotype[i] or ind2.genotype[i]

    offspring = Individual(fitness=None,
                           genotype=list)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def uniform_xover(ind1: Individual, ind2: Individual):
    list1 = [0 for _ in range(NUM_SETS)]
    list2 = [0 for _ in range(NUM_SETS)]

    for i in range(NUM_SETS):
        if random() < 0.5:
            list1[i] = ind1.genotype[i]
            list2[i] = ind2.genotype[i]
        else:
            list1[i] = ind2.genotype[i]
            list2[i] = ind1.genotype[i]

    offspring1 = Individual(fitness=None,
                           genotype=list1)
    offspring2 = Individual(fitness=None,
                           genotype=list2)
    assert len(offspring1.genotype) == NUM_SETS and len(offspring2.genotype) == NUM_SETS
    return (offspring1, offspring2)

In [315]:
def hamming_distance(population):
    total_distance = 0
    num_comp = 0

    for i in range(len(population)):
        for j in range(i+1, len(population)):
            total_distance += sum([x ^ y for x,y in zip(population[i].genotype, population[j].genotype)])
            num_comp += 1

    return total_distance/num_comp

## ES (10)

In [316]:
population = populate()
for epoch in range(100):
    if population[0].fitness == 1:
        break
    
    hd = hamming_distance(population)
    if hd < 400 and MUTATION_PROBABILITY <= 0.85:
        print("INCREASE MUTATION PROBABILITY")
        MUTATION_PROBABILITY += 0.15
    elif hd > 800 and MUTATION_PROBABILITY >= 0.15:
        print("DECREASE MUTATION PROBABILITY")
        MUTATION_PROBABILITY -= 0.15
    
    for generation in range(100):
        if population[0].fitness == 1:
            break

        offspring = list()
        for counter in range(OFFSPRING_SIZE):
            if random() < MUTATION_PROBABILITY:
                p = select_parent(population)
                o = mutate(p, population)
                #o = mutate2(p)

                offspring.append(o)
            else:
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = one_cut_xover(p1, p2)
                #o = my_xover(p1, p2)
                #o1, o2 = uniform_xover(p1, p2)

                offspring.append(o)
                #offspring.append(o1)
                #offspring.append(o2)

        for i in offspring:
            i.fitness = fitness(i.genotype)
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        print(f"{population[0].fitness:.2%}")

print(fitness.calls)

15.92%
16.80%
16.80%
16.80%
22.23%
22.23%
22.23%
22.23%
22.23%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.01%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.40%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
23.79%
INCREASE MUTATION PROBABILITY
23.79%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.18%
24.57%
24.96%
24.96%
24.96%
24.96%
24.96%
24.9

## ES

In [269]:
μ = 10
λ = 20 #number of children
MUTATION_RATE = 0.5
GENERATION_NUM = 5_00 // λ

def es():
    fitness = lab9_lib.make_problem(10)

    population = populate()

    population.sort(key = lambda x:x.fitness, reverse = True)

    parents = population[:μ]

    for _ in range(GENERATION_NUM): #NEW GENERATION
        children = []

        for p in parents:
            children.append(deepcopy(p))

        for _ in range(λ):
            parent = choice(parents)
            new_i = mutate(parent, children)
            new_i.fitness = fitness(new_i.genotype)

            children.append(new_i)

        population = deepcopy(children)

        population.sort(key = lambda x:x.fitness, reverse = True)
        
        parents = population[:μ]
        print(f"{population[0].fitness:.2%}")

    print(fitness.calls)

In [273]:
es()

35.58%
35.58%
41.78%
41.78%
47.33%
47.36%
47.36%
55.34%
55.34%
69.11%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
100.00%
500
